In [113]:
import pandas as pd
import json

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [114]:
dataset = pd.read_csv('description_predictions.csv', index_col=0)
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [115]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

dataset_withoutNAN.reset_index(drop=True, inplace=True)

Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [116]:
"""for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description
"""

In [117]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
0,home team and away team take on each other in ...,0
1,home team begin their title defence with a mat...,1
2,away team will look to make a flying start to ...,2
3,newcomers home team will be aiming to kick off...,2
4,after failing to impress in the 2020/21 serie ...,2


In [118]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range = (2,2), tokenizer = token.tokenize)
text_counts= cv.fit_transform(dataset_withoutNAN.description)

In [119]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, test_size=0.3, random_state=1)

In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mtr


models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [121]:
predictions = {}
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)
    predictions[model_name] = model.predict(X_test)

E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })

E = pd.DataFrame(E).set_index('Model', inplace=False)

In [122]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.675439,0.728956,0.610374,0.611306,0.709647,0.675439,0.641975
Support Vector Machine,0.552632,0.501618,0.444444,0.390323,0.519758,0.552632,0.450792
Multinomial Naive Bayes,0.710526,0.717405,0.653565,0.655152,0.713004,0.710526,0.685058
Decision Tree,0.728070,0.723264,0.689019,0.697018,0.724232,0.728070,0.717430
Random Forest,0.622807,0.833520,0.526646,0.490434,0.780957,0.622807,0.542286
K-Nearest Neighbors,0.570175,0.573016,0.528367,0.512029,0.586685,0.570175,0.548058
